## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tura,IN,2022-02-21 16:05:05,25.5198,90.2201,59.18,56,44,4.97,scattered clouds
1,1,Canavieiras,BR,2022-02-21 16:05:06,-15.6750,-38.9472,87.35,54,18,6.98,few clouds
2,2,Arlit,NE,2022-02-21 16:05:06,18.7369,7.3853,89.15,3,0,5.99,clear sky
3,3,Bredasdorp,ZA,2022-02-21 16:05:06,-34.5322,20.0403,66.27,60,0,17.81,clear sky
4,4,North Bend,US,2022-02-21 16:05:07,43.4065,-124.2243,41.07,99,100,8.05,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Canavieiras,BR,2022-02-21 16:05:06,-15.6750,-38.9472,87.35,54,18,6.98,few clouds
2,2,Arlit,NE,2022-02-21 16:05:06,18.7369,7.3853,89.15,3,0,5.99,clear sky
5,5,Cidreira,BR,2022-02-21 16:05:07,-30.1811,-50.2056,81.84,58,94,4.23,overcast clouds
7,7,Salinopolis,BR,2022-02-21 16:05:08,-0.6136,-47.3561,83.14,76,100,11.81,moderate rain
8,8,Vaini,TO,2022-02-21 16:05:08,-21.2000,-175.2000,78.96,94,40,6.91,scattered clouds
...,...,...,...,...,...,...,...,...,...,...,...
680,680,Carambei,BR,2022-02-21 16:22:45,-24.9178,-50.0972,79.16,50,85,5.35,overcast clouds
681,681,Manokwari,ID,2022-02-21 16:23:45,-0.8667,134.0833,76.68,93,100,4.23,very heavy rain
682,682,Kampot,KH,2022-02-21 16:23:46,10.6167,104.1833,77.05,74,38,3.65,scattered clouds
683,683,Kismayo,SO,2022-02-21 16:23:46,-0.3582,42.5454,79.03,83,58,13.38,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# There are 4 missing values for Country

City_ID                0
City                   0
Country                4
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df=preferred_cities_df.dropna()
# There are a total of 198 cities for the filtered min and max temperatures

In [7]:
clean_cities_df.count()

City_ID                198
City                   198
Country                198
Date                   198
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Canavieiras,BR,87.35,few clouds,-15.6750,-38.9472,
2,Arlit,NE,89.15,clear sky,18.7369,7.3853,
5,Cidreira,BR,81.84,overcast clouds,-30.1811,-50.2056,
7,Salinopolis,BR,83.14,moderate rain,-0.6136,-47.3561,
8,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
...,...,...,...,...,...,...,...
680,Carambei,BR,79.16,overcast clouds,-24.9178,-50.0972,
681,Manokwari,ID,76.68,very heavy rain,-0.8667,134.0833,
682,Kampot,KH,77.05,scattered clouds,10.6167,104.1833,
683,Kismayo,SO,79.03,broken clouds,-0.3582,42.5454,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key 
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
      
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"  
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")  

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

# there are no rows where the Hotel Name is missing

City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file="../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))